# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
import holoviews as hv
from bokeh.io import output_file,save

from bokeh.io import output_file

# Import API key
from api_keys import geoapify_key

os.environ['PROJ_LIB'] = '/Users/steveyuan/anaconda3/envs/python312/lib/python3.12/site-packages/pyproj/proj_dir/share/proj'

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,9.34,88,46,1.79,NZ,1718828878
1,1,adamstown,-25.0660,-130.1015,22.05,74,0,8.43,PN,1718828878
2,2,tiksi,71.6872,128.8694,2.16,83,94,1.53,RU,1718828878
3,3,ta`u,-14.2336,-169.5144,27.32,81,40,7.79,AS,1718828879
4,4,chonchi,-42.6128,-73.8092,6.73,83,78,1.56,CL,1718828879


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# %%capture --no-display
# Fix Err:ImportError: Geographic projection support requires GeoViews, pyproj and cartopy.
# install geoviews pyproj cartopy

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', size=10, color='City',hover_cols=['City'], title="City Locations").opts(width=800, height=400)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

In [8]:
# Save the map to a PNG file (requires selenium and geckodriver)
hv.save(map_plot_1, 'output_data/city_locations_map.png')

In [9]:
# Save the map to an HTML file
output_file("output_data/city_locations_map.html")
save(hv.render(map_plot_1))

'/Users/steveyuan/Documents/github_files/python-api-challenge/output_data/city_locations_map.html'

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter the city with  ideal weather condition：
# 1. max temp between 20 and 28
# 2. humidity between 30 and 50
filtered_data_df = city_data_df[(city_data_df['Max Temp'] <= 28) & (city_data_df['Max Temp'] > 20) & (city_data_df['Humidity'] <= 50) & (city_data_df['Humidity'] > 30)]

# Drop any rows with null values
filtered_data_df = filtered_data_df.dropna(axis=0, how='any')

# Display sample data
filtered_data_df.head()
# filtered_data_df.count()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,college,64.8569,-147.8028,21.36,50,75,0.45,US,1718828893
74,74,inyonga,-6.7167,32.0667,21.46,47,62,2.17,TZ,1718828904
110,110,argo,37.6333,22.7333,27.10,40,0,2.98,GR,1718828917
112,112,congonhas,-20.4997,-43.8578,21.13,48,0,1.36,BR,1718828918
120,120,thunder bay,48.4001,-89.3168,20.95,49,75,6.69,CA,1718828921


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name" to the DataFrame, so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
45,college,US,64.8569,-147.8028,50,
74,inyonga,TZ,-6.7167,32.0667,47,
110,argo,GR,37.6333,22.7333,40,
112,congonhas,BR,-20.4997,-43.8578,48,
120,thunder bay,CA,48.4001,-89.3168,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
# Set the parameters for the type of place
categories = "accommodation.hotel"
radius = 10000
limit = 20

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    # Set the parameters for the current request
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
inyonga - nearest hotel: Mikumi Lodge
argo - nearest hotel: Apollon Hotel
congonhas - nearest hotel: Hotel Vartuli
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
badger - nearest hotel: Clarion Hotel & Suites
hood river - nearest hotel: Hood River Hotel
santa croce camerina - nearest hotel: La Muraglia
cody - nearest hotel: Comfort Inn at Buffalo Bill Village Resort
chyorny yar - nearest hotel: Гостиница
vernon - nearest hotel: Okanagan Royal Park Inn
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
smithers - nearest hotel: Sunshine Inn Hotel
kongolo - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
fort st. john - nearest hotel: Fort St. John Motor Inn
pingzhuang - nearest hotel: No hotel found
haskovo - nearest hotel: aSa Boutique Hotel
inhuma - nearest hotel: No hotel found
ipatovo - nearest hotel: No hotel found
koti

,City,Country,Lat,Lng,Humidity,Hotel Name
45,college,US,64.8569,-147.8028,50,BEST WESTERN PLUS Pioneer Park Inn
74,inyonga,TZ,-6.7167,32.0667,47,Mikumi Lodge
110,argo,GR,37.6333,22.7333,40,Apollon Hotel
112,congonhas,BR,-20.4997,-43.8578,48,Hotel Vartuli
120,thunder bay,CA,48.4001,-89.3168,49,Hampton Inn & Suites by Hilton Thunder Bay
130,badger,US,64.8000,-147.5333,45,Clarion Hotel & Suites
141,hood river,US,45.7054,-121.5215,31,Hood River Hotel
169,santa croce camerina,IT,36.8289,14.5268,41,La Muraglia
195,cody,US,44.5263,-109.0565,31,Comfort Inn at Buffalo Bill Village Resort
207,chyorny yar,RU,48.0624,46.1091,45,Гостиница


In [16]:
# Filter the hotel data, drop the data with no idea hotel
filtered_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

filtered_hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
45,college,US,64.8569,-147.8028,50,BEST WESTERN PLUS Pioneer Park Inn
74,inyonga,TZ,-6.7167,32.0667,47,Mikumi Lodge
110,argo,GR,37.6333,22.7333,40,Apollon Hotel
112,congonhas,BR,-20.4997,-43.8578,48,Hotel Vartuli
120,thunder bay,CA,48.4001,-89.3168,49,Hampton Inn & Suites by Hilton Thunder Bay
130,badger,US,64.8000,-147.5333,45,Clarion Hotel & Suites
141,hood river,US,45.7054,-121.5215,31,Hood River Hotel
169,santa croce camerina,IT,36.8289,14.5268,41,La Muraglia
195,cody,US,44.5263,-109.0565,31,Comfort Inn at Buffalo Bill Village Resort
207,chyorny yar,RU,48.0624,46.1091,45,Гостиница


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_2 = filtered_hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, tiles='OSM', size=10, color='City',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'], title="City Locations with Nearest Hotel"
).opts(width=800, height=400)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)

In [18]:
# Save the map to a PNG file (requires selenium and geckodriver)
hv.save(map_plot_2, 'output_data/hotel_map.png')

In [19]:
# Save the map to an HTML file
output_file("output_data/hotel_map.html")
save(hv.render(map_plot_2))

'/Users/steveyuan/Documents/github_files/python-api-challenge/output_data/hotel_map.html'